In [1]:
import sys
sys.path.append("..")
import datetime

from src.helper import make_ts

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

## 3.1 Simple forecasting methods

### Average Method:

The forecasts of all future values are equal to the average of the historical data:


$\hat{y}_{T+h|T}=\bar{y}=(y_{1}+..+y_{T})/T$

In [2]:
ts_df = make_ts('2000',20)

In [3]:
ts_df.head()

,Observation
Year,
2000-01-01,57.237236
2001-01-01,87.448503
2002-01-01,73.970044
2003-01-01,83.446384
2004-01-01,4.082508


In [60]:
def average_forecast(y, h):
    ts = pd.Series(pd.date_range(str(ts_df.index[-1]), periods = 10, freq=pd.infer_freq(ts_df.index)))

In [25]:
ts_df.index

DatetimeIndex(['2000-01-01', '2001-01-01', '2002-01-01', '2003-01-01',
               '2004-01-01', '2005-01-01', '2006-01-01', '2007-01-01',
               '2008-01-01', '2009-01-01', '2010-01-01', '2011-01-01',
               '2012-01-01', '2013-01-01', '2014-01-01', '2015-01-01',
               '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01'],
              dtype='datetime64[ns]', name='Year', freq=None)

In [7]:
ts_df[-1:]

,Observation
Year,
2019-01-01,37.723546


In [40]:
frequency = pd.infer_freq(ts_df.index)
print(frequency)

AS-JAN


In [41]:
y = pd.Series(pd.date_range(ts_df.index[-1], 10, freq=frequency))

In [42]:
y

Series([], dtype: datetime64[ns])

In [50]:
pd.date_range(str(ts_df.index[-1].year), periods = 10, freq='AS')

'2019'

In [59]:
pd.date_range(str(ts_df.index[-1]), periods = 10, freq=pd.infer_freq(ts_df.index))

DatetimeIndex(['2019-01-01', '2020-01-01', '2021-01-01', '2022-01-01',
               '2023-01-01', '2024-01-01', '2025-01-01', '2026-01-01',
               '2027-01-01', '2028-01-01'],
              dtype='datetime64[ns]', freq='AS-JAN')